**Name:** Jai Gollapudi

**Assignment:** DS4300 HW3

# Data

- I used the `restaurants.json` file provided in the HW3 assignment. 
- The data was imported to Compass.
- The database name is `ds4300_hw3`.
- The collection name is `restaurants`.


# Steps

Please follow these steps to run the program:

1. Import `restaurants.json` into MongoDB Compass.
2. Run the `ds4300_hw3.ipnyb` file.
3. Follow the tutorial and input any of the acceptable inputs in each api call when prompted to do so.

# API and Programmatic Visualizations

In [ ]:
# Importing Libraries
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from datetime import datetime
import plotly.graph_objects as go
import numpy as np

In [ ]:
# Setting max columns for display
pd.set_option('display.max_columns', None)

In [ ]:
class RestaurantsAPI:
    """
    A class to interact with a MongoDB database containing restaurant data, 
    providing methods to query the database and visualize the results.
    """
    
    def __init__(self, uri="mongodb://localhost:27017/"):
        """
        Initializes the API with a connection to the MongoDB database.
        
        :param uri: MongoDB URI string.
        """
        self.client = MongoClient(uri) # Change to your uri if not "mongodb://localhost:27017/"
        self.db = self.client.ds4300_hw3 # 'ds4300_hw3' is the database name
        self.collection = self.db.restaurants # 'restaurants' is the collection name

    def create_geo_index(self):
        """
        Creates a 2dsphere index on the address.coord field for geospatial queries.
        """
        self.collection.create_index([("address.coord", "2dsphere")])

    # API Methods for MongoDB queries

    def find_by_borough(self, borough):
        """
        Finds restaurants in a specific borough and visualizes the top 10 cuisines.
        
        :param borough: The borough to search for restaurants.
        :return: A DataFrame containing the query results.
        """
        # Check if input is in acceptable boroughs
        if borough not in ['Brooklyn', 'Manhattan', 'Queens', 'Bronx', 'Staten Island', 'Missing']:
            print('Borough inputted is not in NYC')
        else:
            cursor = self.collection.find({"borough": borough})
            df = pd.DataFrame(list(cursor))

            # Visualization: Bar chart of top 10 cuisines in the borough
            fig = px.bar(df['cuisine'].value_counts().head(10), 
                         orientation='h', 
                         labels={'index': 'Cuisine', 'value': 'Number of Restaurants'}, 
                         title=f'Top 10 Cuisines in {borough}')
            fig.update_layout(yaxis={'categoryorder':'total ascending'})
            fig.show()

            return df

        
    def find_by_cuisine(self, cuisine):
        """
        Finds restaurants offering a specific cuisine and visualizes distribution by borough.
        
        :param cuisine: The cuisine to search for.
        :return: A DataFrame containing the query results.
        """
        cursor = self.collection.find({"cuisine": cuisine})
        df = pd.DataFrame(list(cursor))

        # Visualization: Bar chart of restaurant distribution by borough
        fig = px.bar(df['borough'].value_counts(), 
                     labels={'index': 'Borough', 'value': 'Number of Restaurants'}, 
                     title=f'Restaurants Offering {cuisine} by Borough')
        fig.update_layout(xaxis={'categoryorder':'total descending'})
        fig.show()

        return df

    
    def find_top_rated(self, min_score):
        """
        Finds restaurants where all grade scores are lower than the specified value and visualizes score distribution.
        :param min_score: The maximum score to filter by (restaurants with a score lower than this are considered top rated).
        :return: A DataFrame containing the filtered query results with scores.
        """
        pipeline = [
            {
                "$project": {
                    "name": 1,
                    "borough": 1,
                    "cuisine": 1,
                    "address": 1,
                    "filtered_grades": {
                        "$filter": {
                            "input": "$grades",
                            "as": "grade",
                            "cond": {"$lt": ["$$grade.score", min_score]}
                        }
                    }
                }
            },
            {"$match": {"filtered_grades.0": {"$exists": True}}},  # Ensure there's at least one grade that meets the condition
        ]
        cursor = self.collection.aggregate(pipeline)
        df = pd.DataFrame(list(cursor))

        # Exploding 'filtered_grades' to access individual scores for visualization
        df_exploded = df.explode('filtered_grades')
        df_exploded['score'] = df_exploded['filtered_grades'].apply(lambda grade: grade['score'] if isinstance(grade, dict) else None)

        # Visualization
        fig = px.histogram(df_exploded, x='score', title='Distribution of Scores for Top Rated Restaurants',
                           nbins=20,  # Adjust bin size as needed
                           labels={'score': 'Score'})
        fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')))
        fig.show()

        return df_exploded

    
    def find_by_zipcode(self, zipcode):
        """
        Finds restaurants in a specific zipcode and visualizes the top 10 cuisines.
        
        :param zipcode: The zipcode to search for restaurants.
        :return: A DataFrame containing the query results.
        """
        cursor = self.collection.find({"address.zipcode": zipcode})
        df = pd.DataFrame(list(cursor))

        # Visualization: Bar chart of top 10 cuisines in the zipcode
        fig = px.bar(df['cuisine'].value_counts().head(10), 
                     orientation='h', 
                     labels={'index': 'Cuisine', 'value': 'Number of Restaurants'}, 
                     title=f'Top 10 Cuisines in Zipcode {zipcode}')
        fig.update_layout(yaxis={'categoryorder':'total ascending'})
        fig.show()

        return df

    
    def aggregate_average_scores(self):
        """
        Aggregates restaurants by name and calculates their average score, visualizing the distribution 
        across dynamically created score ranges in a pie chart.

        :return: A DataFrame containing aggregated results with average scores in descending order.
        """
        # Creating pipeline to aggregate average scores
        pipeline = [
            {"$unwind": "$grades"},
            {"$group": {"_id": "$name", "averageScore": {"$avg": "$grades.score"}}},
            {"$match": {"averageScore": {"$gt": 0}}},  # Excluding scores of 0 or missing
            {"$sort": {"averageScore": 1}},
        ]
        cursor = self.collection.aggregate(pipeline)
        df = pd.DataFrame(list(cursor))

        # Determining max and min scores
        max_score = df['averageScore'].max()
        min_score = df['averageScore'].min()

        # Calculating the range and creating 10 intervals
        score_intervals = np.linspace(min_score, max_score, num=11)

        # Creating labels for the intervals
        labels = [f"{round(score_intervals[i], 2)}-{round(score_intervals[i+1], 2)}" for i in range(len(score_intervals)-1)]

        # Counting the number of restaurants in each interval
        counts = []
        for i in range(len(score_intervals)-1):
            count = df[(df['averageScore'] > score_intervals[i]) & (df['averageScore'] <= score_intervals[i+1])].shape[0]
            counts.append(count)

        # Visualization: Pie chart of average scores distribution across dynamically created ranges
        fig = px.pie(names=labels, values=counts,
                     title='Distribution of Average Scores Across Restaurants',
                     color_discrete_sequence=px.colors.sequential.Plasma, width=1200, height=800)
        
        fig.update_traces(textinfo='percent+label')
        fig.show()

        return df
    

    def count_cuisines_per_borough(self):
        """
        Counts the number of cuisines per borough, visualizing the distribution across boroughs.
        
        :return: A DataFrame containing the count of cuisines per borough.
        """
        # Creating pipeline to group and sort
        pipeline = [
            {"$group": {"_id": {"borough": "$borough", "cuisine": "$cuisine"}, "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ]
        cursor = self.collection.aggregate(pipeline)
        df = pd.DataFrame(list(cursor))

        # Extracting the 'borough' from the '_id' field to create a new column 'borough'
        df['borough'] = df['_id'].apply(lambda x: x['borough'])
        # Extracting the 'cuisine' from the '_id' field to create a new column 'cuisine'
        df['cuisine'] = df['_id'].apply(lambda x: x['cuisine'])
        # Creating a pivot table to transform the DataFrame 
        pivot_df = df.pivot_table(index='borough', columns='cuisine', values='count', fill_value=0)


        # Visualization: Stacked bar chart for count of cuisines per borough
        data = []
        for cuisine in pivot_df.columns:
            data.append(go.Bar(name=cuisine, x=pivot_df.index, y=pivot_df[cuisine]))

        fig = go.Figure(data=data)
        fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'}, 
                          title='Count of Cuisines per Borough',
                          xaxis_title='Borough',
                          yaxis_title='Count of Restaurants',
                          legend_title='Cuisine')
        fig.update_traces(marker=dict(line=dict(width=0.5, color='black')))  # Lines between bars
        fig.show()

        return df

    
    def find_with_geo_near(self, longitude, latitude, max_distance=500):
        """
        Finds restaurants near a specific geographical location within a maximum distance.
        
        :param longitude: Longitude of the location.
        :param latitude: Latitude of the location.
        :param max_distance: Maximum distance from the location (in meters).
        :return: A DataFrame containing restaurants near the specified location.
        """
        cursor = self.collection.find({
            "address.coord": {
                "$near": {
                    "$geometry": {
                        "type": "Point",
                        "coordinates": [longitude, latitude]
                    },
                    "$maxDistance": max_distance
                }
            }
        })
        df = pd.DataFrame(list(cursor))

        # Checking if the DataFrame is not empty and contains 'address' column and 'coord'
        if not df.empty and 'address' in df.columns and 'coord' in df.iloc[0]['address']:
            # Extracting longitude from 'coord' inside the 'address' column and creating a new 'longitude' column
            df['longitude'] = df['address'].apply(lambda x: x['coord'][0] if 'coord' in x else None)
            # Extracting latitude from 'coord' inside the 'address' column and creating a new 'latitude' column
            df['latitude'] = df['address'].apply(lambda x: x['coord'][1] if 'coord' in x else None)

            # Visualization: Scatter plot for restaurants near location
            
            fig = px.scatter(df, x='longitude', y='latitude',
                             hover_name='name', hover_data=['cuisine'],
                             title='Restaurants Near Location')
            # Adding the specific coordinate as a red dot on the map
            fig.add_trace(go.Scatter(
                x=[longitude], y=[latitude], 
                mode='markers', 
                marker=dict(color='red', size=15, line=dict(width=2, color='DarkRed')),
                name='Search Point'
            ))
            fig.update_traces(marker=dict(size=12, opacity=0.8, line=dict(width=2, color='DarkSlateGrey')))
            fig.update_layout(title=f'Restaurants within {max_distance}m of [{latitude}, {longitude}]',
                              xaxis_title="Longitude",
                              yaxis_title="Latitude")
            fig.update_layout(hoverlabel=dict(bgcolor="white", font_size=12, font_family="Rockwell"))
            fig.show()

        else:
            print("No data found or 'address.coord' is missing.")

        return df
    

    def filter_by_grade(self, grade):
        """
        Finds restaurants with a specific grade and visualizes the distribution of these restaurants by cuisine.
        
        :param grade: The grade to filter restaurants by.
        :return: A DataFrame containing restaurants filtered by the specified grade.
        """
        cursor = self.collection.find({"grades.grade": grade})
        df = pd.DataFrame(list(cursor))
        
        # Counting of restaurants by cuisine
        cuisine_grade_distribution = df['cuisine'].value_counts().head(10)
        
        # Visualization: Pie chart showing distribution of restaurants by cuisine
        fig = px.pie(cuisine_grade_distribution, 
                     values=cuisine_grade_distribution.values, 
                     names=cuisine_grade_distribution.index,
                     title=f'Distribution of {grade} Grade Restaurants by Cuisine (Top 10)',
                     hole=0.5, width = 1200, height = 800)  # Creates a donut-like pie chart
        fig.update_traces(textinfo='percent+label', pull=[0.1] * 10)  # Customizing text info and pulling out slices
        fig.show()

        return df

    
    def grades_between_dates(self, start_date, end_date, rows=100):
        """
        Retrieves and visualizes the distribution of restaurant health inspection grades between two dates.

        :param start_date (datetime): The start date for filtering grades.
        :param end_date (datetime): The end date for filtering grades.
        :param rows (int): The maximum number of records to return. Defaults to 100.
        :return: A DataFrame containing the restaurants and their grades within the specified date range.
        """
        cursor = self.collection.find({
            "grades.date": {
                "$gte": start_date,
                "$lte": end_date
            }
        }).limit(rows)
        df = pd.DataFrame(list(cursor))

        if not df.empty:
            # Exploding the grades array to work with individual grade entries
            df = df.explode('grades')
            # Converting grades.date to datetime, and filtering the DataFrame based on the date range
            df['grade_date'] = pd.to_datetime(df['grades'].apply(lambda x: x['date']))
            df = df[(df['grade_date'] >= start_date) & (df['grade_date'] <= end_date)]

            # Ensuring the DataFrame is sorted by date
            df.sort_values('grade_date', inplace=True)

            # Counting grades by date within the specified range
            grade_counts = df['grade_date'].value_counts().sort_index()

            # Visualization: Line chart showing trend of grades between the specified dates
            fig = px.line(grade_counts, 
                          x=grade_counts.index, 
                          y=grade_counts.values, 
                          title=f'Trend of Grades Between {start_date.strftime("%Y-%m-%d")} and {end_date.strftime("%Y-%m-%d")}')
            fig.update_layout(xaxis_title='Date', 
                              yaxis_title='Number of Grades Given',
                              xaxis=dict(tickformat='%Y-%m-%d'))
            fig.show()
        else:
            print("No grades found in the specified date range.")

        return df

    
    def sort_by_name(self):
        """
        Sorts restaurants by their name alphabetically.
        
        :return: A DataFrame containing restaurants sorted by name.
        """
        cursor = self.collection.find({"name": {"$nin": [None, "", " "]}}).sort("name", 1)
        df = pd.DataFrame(list(cursor))

        # Further filtering out any rows where the name is NaN, missing, or contains only whitespace 
        df = df[df['name'].str.strip().astype(bool)]

        return df

# Tutorial

The API above provides 10 unique methods used to answer 10 unique questions about the restaurants dataset:

- **3.0.1:**  Find restaurants in a specific borough 
- **3.0.2:**  Find all restaurants offering a specific cuisine
- **3.0.3:**  Find restaurants with a score lower than a specified value (indicating high quality)
- **3.0.4:**  Find all restaurants in a specific zipcode
- **3.0.5:**  Aggregate and average scores of restaurants, sorted by average score
- **3.0.6:**  Count cuisines per borough
- **3.0.7:**  Find restaurants near a specific geographical location
- **3.0.8:**  Filter restaurants by a specific grade
- **3.0.9:**  Find restaurants with grades assigned during a specific speriod
- **3.0.10:**  Sort restaurants by name

In [ ]:
# Instantiating the API
api = RestaurantsAPI()
# Calling the create_geo_index method
api.create_geo_index()

### Find restaurants in a specific borough 

**Acceptable Inputs:** 
- **borough:** [Brooklyn, Manhattan, Queens, Bronx, Staten Island]
<br/> 

**Output:**
- **Visualization:** A horizontal bar chart of the top 10 cuisines in a particular borough
- **Data:** A dataframe of all the resturants in a particular borough

In [ ]:
borough = input("Enter a Borough: ")
print(f'Restaurants in {borough}: ')
api.find_by_borough(borough)

**Interpretation:**

The bar chart shows the variety and prevalence of different types of cuisine available in one of NYC's borough's restaurants. Typically, american cuisine is the most common, indicating a strong preference or cultural influence in the area, followed by Italian, which also enjoys significant popularity. The presence of diverse cuisines like Irish, French, and Spanish to a lesser extent reflects NYC's cultural diversity and the wide array of dining options available to residents and visitors.

### Find all restaurants offering a specific cuisine

**Acceptable Inputs:** 
- **cuisine:** [American, Italian, Indian, Japanese, Chinese, Pizza, Mexican, Irish, Russian]
<br/> 

**Output:**
- **Visualization:** A vertical bar chart of the distribution of restaurants of a particular cuisine in each borough
- **Data:** A dataframe of all the resturants serving a particular cusuine

In [ ]:
cuisine = input("Enter a Cuisine: ")
print(f'Restaurants offering {cuisine}: ')
api.find_by_cuisine(cuisine)

**Interpretation:**
    
This bar chart presents a comparative view of the distribution of restaurants offering a selected cuisine across different boroughs. The chart reveals disparities in restaurant prevalence, with some boroughs showing a higher concentration of this cuisine than others. Such variations could reflect demographic trends, cultural preferences, or economic factors influencing the culinary landscape of the city. 

NOTE: The 'Missing' category may indicate data incompleteness or unclassified locations.

### Find restaurants with a score lower than a specified value (indicating high quality)

**Acceptable Inputs:** 
- **score:** [Any positive integer from 0-75]
<br/> 

**Output:**
- **Visualization:** A histogram chart of the distribution of top rated restaurants 
- **Data:** A dataframe of all the resturants with a score less than the inputted score

In [ ]:
score = int(input("Enter a Score: "))
print(f'Top Rated Restaurants (score < {score}):')
api.find_top_rated(score)

**Interpretation:**

The histogram illustrates the distribution of scores for top-rated restaurants, reflecting health inspection results. The majority of restaurants are clustered within the lower score range, suggesting a high level of compliance with standards. A smaller count of restaurants have higher scores, which may indicate areas for improvement. This distribution could be used by restaurant owners to benchmark their performance and by customers to inform their dining choices.

### Find all restaurants in a specific zipcode

**Acceptable Inputs:** 
- **zipcode:** [10019, 10014, 10013, 10022, 10036]
<br/> 

**Output:**
- **Visualization:** A horizontal bar chart of the distribution of resturants in a zip code by top cuisine. 
- **Data:** A dataframe of all the resturants in a particular zip code

In [ ]:
zipcode = input("Enter a zipcode: ")
print(f'Restaurants in Zipcode {zipcode}:')
api.find_by_zipcode(zipcode)

**Interpretation:**

The chart illustrates the diversity of restaurant cuisines in a specific area, showcasing a broad culinary landscape. The prominence of American and Italian cuisines suggests these are popular dining choices, reflecting either a cultural preference or a larger number of establishments serving these foods. The presence of a variety of other cuisines, including French, Japanese, and Mexican, indicates a rich, cosmopolitan dining scene that caters to a range of tastes and preferences, potentially driven by the demographic makeup and culinary interests of the local population.

### Aggregate and average scores of restaurants, sorted by average score (ascending)


**Acceptable Inputs:** N/A
<br/> 

**Output:**
- **Visualization:** A pie chart of the distribution of scores across restaurants
- **Data:** A dataframe of all the restaurants with their average scores in ascending order

In [ ]:
print("Average Scores of Restaurants in Descending order:")
api.aggregate_average_scores()

 **Interpretation:**

The pie chart illustrates the distribution of average health inspection scores across different restaurants. The largest segment represents restaurants with scores ranging from 8.1 to 15.53, encompassing the majority at 65.2%, indicating that most establishments fall within this score range. Notably, there is a significant proportion (19.7%) of restaurants scoring between 0.67 to 8.1, which suggests a noteworthy number of restaurants perform exceptionally well in health inspections. The presence of smaller segments indicates fewer restaurants with higher scores, reflecting a trend towards better health standards among the majority of these restaurants.

### Count cuisines per borough

**Acceptable Inputs:** N/A
<br/> 

**Output:**
- **Visualization:** A stacked bar chart of all the cuisines in each borough
- **Data:** A dataframe of the count of the number of resturants of each cuisine in each borough


In [ ]:
print("Count of Cuisines per Borough:")
api.count_cuisines_per_borough()

**Interpretation:**

The stacked bar chart "Count of Cuisines per Borough" provides a visual representation of the variety of cuisines available in each of New York City's boroughs. The chart's multicolored layers illustrate the rich diversity of culinary options, revealing not only the quantity but also the food choices. The relatively larger height of the Manhattan bar for American food suggests that other areas might be more diverse in terms of options.

### Find restaurants near a specific geographical location

**Acceptable Inputs:** 
- **coordinates:** (-73.856077, 40.848447), (-73.7796156, 40.7298006), (-73.78309349999999, 40.8374725) **[Note: Don't 
include the parentheses]**
- **distance:** Any positive integer from 10 to 1000 (restricting to 1000m for better visual appearance)
<br/> 

**Output:**
- **Visualization:** A scatter plot of the resturant within a specified distance to a coordinate
- **Data:** A dataframe of all the resturants within a specified distance to a coordinate


In [ ]:
coordinates = (input("Enter coordinates : ")).split(', ')
distance = int(input("Enter a distance: ")) 
latitude = float(coordinates[0])
longitude = float(coordinates[1])
api.find_with_geo_near(latitude, longitude, distance)

**Interpretation:**

The scatter plot shows the geographic distribution of restaurants in proximity to a specified coordinate location, marked as "Search Point" in red. The visualization enables one to quickly assess the density and clustering of dining options within a certain area, which could be useful for someone looking to explore the culinary scene nearby. The red marker effectively highlights the reference point, providing a clear anchor for spatial orientation on the map.

### Filter restaurants by a specific grade

**Acceptable Inputs:** 
- **grade:** [A, B, C]
<br/> 

**Output:**
- **Visualization:** A pie chart showing the distribution of a particular grade of restaurants by cuisine
- **Data:** A dataframe of each restaurant of a particular grade


In [ ]:
grade = input("Enter a grade: ").upper()
print(f'Restaurants with Grade {grade}:')
api.filter_by_grade(grade)

**Interpretation:**

The pie chart displays the proportions of top-rated restaurants by cuisine type. The variety of cuisines represented, including Italian, Mexican, and Japanese, showcases the diverse array of high-quality dining options available to consumers. The chart also suggests the broad palate preferences of the area's patrons, who value a mix of both traditional and international flavors.

### Find restaurants with grades assigned during a specific speriod 

**Acceptable Inputs:** 
- **start_date:** Any date after 2011-02-15 in YYYY-MM-DD format
- **end_date:** Any date before 2015-01-20 in YYYY-MM-DD format
<br/> 

**Output:**
- **Visualization:** A line chart showing the number of restaurants awarded a grade between specific dates **(Note: Limiting to 100 rows for better visual appearance)**
- **Data:** A dataframe of each restaurant awarded a grade between specific dates 

In [ ]:
start_date = input("Enter a start date (YYYY-MM-DD): ")
end_date = input("Enter an end date (YYYY-MM-DD): ")
print(f'Restaurants with Recent Grades between {start_date} and {end_date}:')

# Convert input strings to datetime objects
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

api.grades_between_dates(start_date_dt, end_date_dt)

**Interpretation:**

The line chart displays the frequency of grades awarded to restaurants over a specific period. The plot shows fluctuations in the number of grades given on different dates, with some peaks suggesting days on which a higher number of inspections occurred or a higher volume of grades were assigned. The overall trend could indicate varying levels of inspection activity over time or could reflect seasonal patterns in inspection schedules. This information might be useful for understanding the inspection processes or for restaurants preparing for periods of heightened inspection activity.

### Sort restaurants by name

**Acceptable Inputs:** N/A
<br/> 

**Output:**
- **Visualization:** N/A
- **Data:** A dataframe of each restaurant sorted in descending order by name


In [ ]:
print("\nRestaurants Sorted by Name:")
api.sort_by_name()

**Interpretation:**

The output shows a DataFrame with restaurant data sorted alphabetically by the restaurant's name. The data includes unique identifiers, addresses with geocoordinates, borough locations, cuisine types, health grades, and restaurant names. 